In [1]:
import os
import copy
import time
import torch
import skimage
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
print("complete")

complete


In [2]:
train_vectors = np.load("train_vectors_car.npy")
test_vectors = np.load("test_vectors_car.npy")

In [3]:
def pro_select(vectors, probs, num):
    points = np.zeros((num, 2))
    core_point = np.zeros((1, 2))
    for i in range(num):
        x = random.uniform(0, 1)
        cumprob = 0.0
        k = -1
        while x > cumprob:
            k += 1
            cumprob += probs[k]
        points[i, 0], points[i, 1] = vectors[k, 0], vectors[k, 1]
        
        adj_list = []
        core_point[0, 0], core_point[0, 1] = vectors[k, 0], vectors[k, 1]
        dis = np.sum(pow(vectors - core_point, 2), axis=1)
        for j in range(len(dis)):
            if dis[j] <= 8:
                adj_list.append(j)
        for j in range(len(adj_list)):
            probs[adj_list[j]] /= 100.0
            
        probs[k] = 0
        probs = probs/np.sum(probs)
        
    return points

class Bird_Test_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, train_vectors, num, transform=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform
        self.train_vectors = train_vectors
        self.part_num = num

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
            
        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448))) 
        label = int(img_dir_label[1])
        sample = {}
        
        vectors = self.train_vectors[idx]

        points = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)

        points1 = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)

        points2 = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)

        for i in range(self.part_num):
            x, y = points[i][0] * 16 + 8, points[i][1] * 16 + 8
            l = random.randint(46, 98)
#             l = 46
            m1, m2, n1, n2 = int(np.maximum(0, x - l)), int(np.minimum(448, x + l)), \
                            int(np.maximum(0, y - l)), int(np.minimum(448, y + l))
            sample['image%d' % i] = image[m1:m2, n1:n2]
            x1, y1 = points1[i][0] * 16 + 8, points1[i][1] * 16 + 8
            l1 = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x1 - l1)), int(np.minimum(448, x1 + l1)), \
                            int(np.maximum(0, y1 - l1)), int(np.minimum(448, y1 + l1))
            sample['s_image%d' % i] = image[m1:m2, n1:n2]
            x2, y2 = points2[i][0] * 16 + 8, points2[i][1] * 16 + 8
            l2 = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x2 - l2)), int(np.minimum(448, x2 + l2)), \
                            int(np.maximum(0, y2 - l2)), int(np.minimum(448, y2 + l2))
            sample['t_image%d' % i] = image[m1:m2, n1:n2]
            
        sample['label'] = label

        if self.transform:
            for i in range(self.part_num):
                sample['image%d' % i] = self.transform(sample['image%d' % i])
                sample['s_image%d' % i] = self.transform(sample['s_image%d' % i])
                sample['t_image%d' % i] = self.transform(sample['t_image%d' % i])

        return sample
    
class Bird_Dataset_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, train_vectors, num, transform=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform
        self.train_vectors = train_vectors
        self.part_num = num

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
            
        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448))) 
        label = int(img_dir_label[1])
        sample = {}
        
        vectors = self.train_vectors[idx]
        points = pro_select(vectors[:, :2].astype(int), vectors[:, 3], self.part_num)
        for i in range(self.part_num):
            x, y = points[i][0] * 16 + 8, points[i][1] * 16 + 8
            l = random.randint(46, 98)
#             l = 46
            m1, m2, n1, n2 = int(np.maximum(0, x - l)), int(np.minimum(448, x + l)), \
                            int(np.maximum(0, y - l)), int(np.minimum(448, y + l))
            sample['image%d' % i] = image[m1:m2, n1:n2]
        
        sample['label'] = label

        if self.transform:
            for i in range(self.part_num):
                sample['image%d' % i] = self.transform(sample['image%d' % i])

        return sample

In [4]:
import random
data_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(224),
                transforms.RandomCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.471, 0.460, 0.455], [0.292, 0.291, 0.299])
            ])

root_dir = '/home/xinsir/Pytorch/car_data'
train_file = '/home/xinsir/Pytorch/car_data/train_list.txt'
test_file = '/home/xinsir/Pytorch/car_data/test_list.txt'

img_data = {'train': Bird_Dataset_Process(train_file, root_dir, train_vectors, 4, data_transform),
                  'test': Bird_Test_Process(test_file, root_dir, test_vectors, 4, data_transform)}
loader_img = {x: torch.utils.data.DataLoader(img_data[x], batch_size=20,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

dataset_sizes = {x: len(img_data[x]) for x in ['train', 'test']}
print(img_data['test'][0])

/home/xinsir/.tensorflow/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/xinsir/.tensorflow/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/xinsir/.tensorflow/lib/python3.5/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


{'image1': tensor([[[-0.2297, -0.2700, -0.3237,  ...,  0.5224,  0.5224,  0.5224],
         [-0.2969, -0.3237, -0.3775,  ...,  0.5224,  0.5224,  0.5224],
         [-0.3775, -0.3909, -0.4312,  ...,  0.5224,  0.5224,  0.5224],
         ...,
         [ 0.0926,  0.1195, -0.0283,  ...,  0.7238,  0.7238,  0.7104],
         [-0.0551,  0.0389, -0.0417,  ...,  0.7238,  0.7238,  0.7104],
         [-0.2297, -0.1223, -0.1223,  ...,  0.7238,  0.7238,  0.7104]],

        [[-0.1927, -0.2197, -0.2601,  ...,  0.7102,  0.7102,  0.7102],
         [-0.2601, -0.2736, -0.3140,  ...,  0.7102,  0.7102,  0.7102],
         [-0.3409, -0.3544, -0.3814,  ...,  0.7102,  0.7102,  0.7102],
         ...,
         [-0.0579, -0.0445, -0.1927,  ...,  0.7911,  0.7911,  0.7776],
         [-0.2062, -0.1253, -0.2062,  ...,  0.7911,  0.7911,  0.7776],
         [-0.3949, -0.2870, -0.2870,  ...,  0.7911,  0.7911,  0.7911]],

        [[-0.1315, -0.1446, -0.1839,  ...,  1.1276,  1.1276,  1.1276],
         [-0.1971, -0.2102, -0.236

In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, resume=None):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    start_epoch = 0
    if resume:
        ckpt = torch.load(resume)
        model = ckpt['best_model']
        optimizer = ckpt['optimizer']
        scheduler = ckpt['scheduler']       
        start_epoch = ckpt['epoch']
        
    for epoch in range(start_epoch, num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            start_time = time.time()        
            if epoch % 5 == 0 and phase == 'train':
                for param_group in optimizer.param_groups:
                    lr = param_group['lr']
                    print("***********************")
                    print("learning rate = %f" % lr)
            
            # Iterate over data.
            for i_batch, sample_batched in enumerate(loader_img[phase]):
                img0 = sample_batched['image0']
                img1 = sample_batched['image1']
                img2 = sample_batched['image2']
                img3 = sample_batched['image3']
                labels = sample_batched['label']
                if phase == 'test':
                    s_img0 = sample_batched['s_image0']
                    s_img1 = sample_batched['s_image1']
                    s_img2 = sample_batched['s_image2']
                    s_img3 = sample_batched['s_image3']
                    t_img0 = sample_batched['t_image0']
                    t_img1 = sample_batched['t_image1']
                    t_img2 = sample_batched['t_image2']
                    t_img3 = sample_batched['t_image3']
                    s_img0 = s_img0.to(device)
                    s_img1 = s_img1.to(device)
                    s_img2 = s_img2.to(device)
                    s_img3 = s_img3.to(device)
                    t_img0 = t_img0.to(device)
                    t_img1 = t_img1.to(device)
                    t_img2 = t_img2.to(device)
                    t_img3 = t_img3.to(device)
                    
                img0 = img0.to(device)
                img1 = img1.to(device)
                img2 = img2.to(device)
                img3 = img3.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train':
                        output0 = model(img0)
                        output1 = model(img1)
                        output2 = model(img2)
                        output3 = model(img3)
                    else:
                        output0 = (model(img0) + model(s_img0) + model(t_img0))/3.0
                        output1 = (model(img1) + model(s_img1) + model(t_img1))/3.0
                        output2 = (model(img2) + model(s_img2) + model(t_img2))/3.0
                        output3 = (model(img3) + model(s_img3) + model(t_img3))/3.0
                    outputs = output0 + output1 + output2 + output3
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * img3.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                if i_batch % 50 == 0 and phase == 'train':
                    print("Iteration %d, loss = %f" % (i_batch, loss))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'test':
                scheduler.step(epoch_loss)
                
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            cost_time = (time.time() - start_time)/60.0
            print('{} 1 epoch time: {:.2f}min'.format(
                phase, cost_time))
            
            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({
                    'epoch': epoch,
                    'test_loss': epoch_loss,
                    'test_acc': epoch_acc,
                    'optimizer': optimizer,
                    'scheduler': scheduler,
                    'best_model': model},
                    os.path.join('/home/xinsir/Pytorch/random_car/', '%03d.ckpt' % epoch))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [6]:
import time
import copy
import warnings
import torch.optim as optim
from torch.optim import lr_scheduler

warnings.filterwarnings('ignore')

device = torch.device("cuda:0")

vgg_16 = models.vgg16(pretrained=True)
vgg_16.classifier = nn.Sequential(*list(vgg_16.classifier.children())[:1])
vgg_16.features[30] = nn.AvgPool2d(kernel_size=14, stride=1, padding=0)
vgg_16.features.add_module('31', nn.Dropout(0.8)) 
vgg_16.classifier[0] = nn.Linear(in_features=512, out_features=200, bias=True)
# vgg_16 = nn.DataParallel(vgg_16, device_ids=[2, 3])
vgg_16 = vgg_16.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(vgg_16.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='min', factor=0.5, patience=5)

model = train_model(vgg_16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=90)
                    # resume='/home/xinsir/Pytorch/random_car/006.ckpt')

Epoch 0/89
----------
2018-12-21 14:22:08
----------
***********************
learning rate = 0.001000
Iteration 0, loss = 5.410166
Iteration 50, loss = 5.321151
Iteration 100, loss = 5.352105
Iteration 150, loss = 5.310679
Iteration 200, loss = 5.194160
Iteration 250, loss = 5.348280
Iteration 300, loss = 5.237159
Iteration 350, loss = 5.301087
Iteration 400, loss = 5.192891
train Loss: 5.2982 Acc: 0.0081
train 1 epoch time: 5.34min
test Loss: 5.2051 Acc: 0.0179
test 1 epoch time: 4.91min

Epoch 1/89
----------
2018-12-21 14:32:23
----------
Iteration 0, loss = 5.225879
Iteration 50, loss = 5.316267
Iteration 100, loss = 5.251706
Iteration 150, loss = 5.211287
Iteration 200, loss = 5.503820
Iteration 250, loss = 5.191754
Iteration 300, loss = 4.858147
Iteration 350, loss = 4.958974
Iteration 400, loss = 4.896089
train Loss: 5.1093 Acc: 0.0236
train 1 epoch time: 5.35min
test Loss: 4.7752 Acc: 0.0669
test 1 epoch time: 4.88min

Epoch 2/89
----------
2018-12-21 14:42:37
----------
Iterat

Iteration 0, loss = 0.747882
Iteration 50, loss = 0.954252
Iteration 100, loss = 0.456934
Iteration 150, loss = 0.875759
Iteration 200, loss = 1.107138
Iteration 250, loss = 0.651311
Iteration 300, loss = 0.792954
Iteration 350, loss = 1.192148
Iteration 400, loss = 1.068950
train Loss: 0.7682 Acc: 0.7797
train 1 epoch time: 5.28min
test Loss: 0.6699 Acc: 0.8261
test 1 epoch time: 4.71min

Epoch 19/89
----------
2018-12-21 17:35:12
----------
Iteration 0, loss = 0.702150
Iteration 50, loss = 1.231266
Iteration 100, loss = 0.536128
Iteration 150, loss = 0.494306
Iteration 200, loss = 0.941927
Iteration 250, loss = 0.540256
Iteration 300, loss = 0.917713
Iteration 350, loss = 0.435223
Iteration 400, loss = 0.724699
train Loss: 0.7193 Acc: 0.7876
train 1 epoch time: 5.28min
test Loss: 0.7249 Acc: 0.8118
test 1 epoch time: 4.71min

Epoch 20/89
----------
2018-12-21 17:45:11
----------
***********************
learning rate = 0.001000
Iteration 0, loss = 0.408635
Iteration 50, loss = 0.79318

Iteration 0, loss = 0.254793
Iteration 50, loss = 0.587014
Iteration 100, loss = 0.423203
Iteration 150, loss = 0.225402
Iteration 200, loss = 0.456930
Iteration 250, loss = 0.606910
Iteration 300, loss = 0.352579
Iteration 350, loss = 0.592662
Iteration 400, loss = 0.491774
train Loss: 0.3415 Acc: 0.8962
train 1 epoch time: 5.26min
test Loss: 0.5561 Acc: 0.8590
test 1 epoch time: 4.67min

Epoch 37/89
----------
2018-12-21 20:34:21
----------
Iteration 0, loss = 0.322195
Iteration 50, loss = 0.440963
Iteration 100, loss = 0.105577
Iteration 150, loss = 0.455993
Iteration 200, loss = 0.433855
Iteration 250, loss = 0.284868
Iteration 300, loss = 0.146530
Iteration 350, loss = 0.208372
Iteration 400, loss = 0.099165
train Loss: 0.3404 Acc: 0.8972
train 1 epoch time: 5.27min
test Loss: 0.5265 Acc: 0.8709
test 1 epoch time: 4.68min

Epoch 38/89
----------
2018-12-21 20:44:18
----------
Iteration 0, loss = 0.477413
Iteration 50, loss = 0.536494
Iteration 100, loss = 0.230595
Iteration 150, l

Iteration 0, loss = 0.013343
Iteration 50, loss = 0.012587
Iteration 100, loss = 0.018813
Iteration 150, loss = 0.004026
Iteration 200, loss = 0.035380
Iteration 250, loss = 0.367415
Iteration 300, loss = 0.020481
Iteration 350, loss = 0.057164
Iteration 400, loss = 0.180867
train Loss: 0.1039 Acc: 0.9668
train 1 epoch time: 5.26min
test Loss: 0.4962 Acc: 0.8975
test 1 epoch time: 4.67min

Epoch 55/89
----------
2018-12-21 23:33:19
----------
***********************
learning rate = 0.000250
Iteration 0, loss = 0.135867
Iteration 50, loss = 0.412352
Iteration 100, loss = 0.017862
Iteration 150, loss = 0.562416
Iteration 200, loss = 0.002212
Iteration 250, loss = 0.066412
Iteration 300, loss = 0.074483
Iteration 350, loss = 0.089425
Iteration 400, loss = 0.217265
train Loss: 0.1023 Acc: 0.9686
train 1 epoch time: 5.26min
test Loss: 0.4802 Acc: 0.8969
test 1 epoch time: 4.68min

Epoch 56/89
----------
2018-12-21 23:43:15
----------
Iteration 0, loss = 0.259237
Iteration 50, loss = 0.02431

Iteration 0, loss = 0.042829
Iteration 50, loss = 0.013095
Iteration 100, loss = 0.041023
Iteration 150, loss = 0.214758
Iteration 200, loss = 0.008015
Iteration 250, loss = 0.042670
Iteration 300, loss = 0.003710
Iteration 350, loss = 0.097900
Iteration 400, loss = 0.258823
train Loss: 0.0755 Acc: 0.9762
train 1 epoch time: 5.27min
test Loss: 0.4742 Acc: 0.9036
test 1 epoch time: 4.68min

Epoch 73/89
----------
2018-12-22 02:32:21
----------
Iteration 0, loss = 0.022643
Iteration 50, loss = 0.101100
Iteration 100, loss = 0.054008
Iteration 150, loss = 0.081970
Iteration 200, loss = 0.018893
Iteration 250, loss = 0.050054
Iteration 300, loss = 0.043128
Iteration 350, loss = 0.012094
Iteration 400, loss = 0.008237
train Loss: 0.0647 Acc: 0.9815
train 1 epoch time: 5.26min
test Loss: 0.4770 Acc: 0.9056
test 1 epoch time: 4.68min

Epoch 74/89
----------
2018-12-22 02:42:18
----------
Iteration 0, loss = 0.046574
Iteration 50, loss = 0.124376
Iteration 100, loss = 0.207913
Iteration 150, l

In [8]:
torch.save(model, '/home/xinsir/Pytorch/random_part_car')